In [9]:
import streamlit as st
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [11]:
df = pd.read_csv("Export_TER_juin2023_FIX_SafwanChendeb.csv",sep=";")

DATA CLEAN

Enlever les valeurs nulles ou trop petites de Largeur de bande

In [18]:
df2 = df.dropna(subset=['ASS_LGBD_KHZ'])
df2 = df2[df2["ASS_LGBD_KHZ"] >1000]


Enlever les antennes qui ont été supprimées

In [ ]:
enreg_to_supp = df2[df2['MVT_CODE'] == 'SUP']['N° ENREG']
df2 = df2[~df2['N° ENREG'].isin(enreg_to_supp)]


Garder seulement les ajouts d'antenne

In [13]:
df2  = df2[df2['MVT_CODE'] == 'ADD']

Enlever les MOD

In [19]:
df2 = df2[df2['MVT_CODE'] != 'MOD' ]

Garder seulement les dernières modifications de chaque antenne

In [ ]:
df2['Date CAF'] = pd.to_datetime(df['Date CAF'], format='%d/%m/%Y')
df2 = df2.sort_values('Date CAF').drop_duplicates(subset = "N° ENREG", keep = 'last')

TEST

In [15]:
df2.head(8)

,N° ENREG,Code CAF,ASS_FRQ_KHZ,ASS_LGBD_KHZ,TER_ANT_AZM_MAX,TER_ANT_ANG,PT_LONG_DEG,PT_LONG_MIN,PT_LONG_SEC,PT_LONG_ORIENT,PT_LAT_DEG,PT_LAT_MIN,PT_LAT_SEC,PT_LAT_ORIENT,Date CAF,BASE,MVT_CODE
2,1428858.0,K,26145000,56000.0,320.7,2.6,7,3,5,E,43,37,2,N,18/05/2017,H,ADD
3,1502495.0,K,6920000,40000.0,307.4,3.0,6,36,33,E,47,38,35,N,11/04/2019,R,ADD
5,1455698.0,K,6460000,40000.0,247.6,2.5,5,38,38,E,43,41,17,N,22/02/2018,H,ADD
10,1277365.0,K,13087000,28000.0,149.3,4.7,2,25,38,E,43,47,44,N,18/12/2014,H,ADD
17,1083711.0,K,6660000,40000.0,188.8,2.5,1,48,34,E,50,4,53,N,20/04/2010,H,ADD
22,1632137.0,K,19150000,110000.0,30.1,1.2,2,39,41,E,47,41,41,N,17/03/2022,R,ADD
29,1659722.0,K,81375000,500000.0,247.7,0.8,0,21,50,W,46,14,36,N,16/03/2023,R,ADD
32,1661516.0,K,19480000,110000.0,253.7,1.2,2,30,44,E,49,54,15,N,22/06/2023,R,ADD


GRAPH DU NOMBRE D'AJOUTS D'ANTENNES PAR MOIS

Création d'une nouvelle variable 'YearMonth' pour trier la création d'antenne par mois

In [16]:

df2['Date CAF'] = pd.to_datetime(df2['Date CAF'], format='%d/%m/%Y')
df2['YearMonth'] = df2['Date CAF'].dt.to_period('M')
monthly_counts = df2['YearMonth'].value_counts().sort_index().reset_index()
monthly_counts.columns = ['YearMonth', 'Number of Antennas']
monthly_counts['YearMonth'] = monthly_counts['YearMonth'].astype(str)


Plot du nombre d'ajout d'antennes par mois

In [17]:

fig = px.line(
    monthly_counts, 
    x='YearMonth', 
    y='Number of Antennas', 
    title='Évolution du nombre d\'ajout d\'antennes par mois',
    markers=True,
    labels={'YearMonth': 'Mois', 'Number of Antennas': 'Nombre d\'antennes ajoutées'}
)

fig.update_layout(
    xaxis_title='Mois',
    yaxis_title='Nombre d\'antennes ajoutées',
    xaxis_tickangle=-45
)

fig.write_html('monthly_antenna_additions.html')

GRAPH DU NOMBRE D'ANTENNES EN SERVICE PAR MOIS

Création d'un nouveau DatFrame contenant le nombre d'ajout et de suppression d'antennes par mois

In [20]:
df2['Date CAF'] = pd.to_datetime(df2['Date CAF'], format='%d/%m/%Y')
df2 = df2.sort_values(by='Date CAF')
df2['YearMonth'] = df2['Date CAF'].dt.to_period('M')
adds_per_month = df2[df2['MVT_CODE'] == 'ADD'].groupby('YearMonth').size()
sups_per_month = df2[df2['MVT_CODE'] == 'SUP'].groupby('YearMonth').size()

Calcul du nombre d'antennes en service par mois

In [22]:
service_df_corrected = pd.DataFrame({'Adds': adds_per_month, 'Sups': sups_per_month}).fillna(0)

service_df_corrected['Cumulative Adds'] = service_df_corrected['Adds'].cumsum()
service_df_corrected['Cumulative Sups'] = service_df_corrected['Sups'].cumsum()

service_df_corrected['Antennas in Service'] = service_df_corrected['Cumulative Adds'] - service_df_corrected['Cumulative Sups']

service_df_corrected = service_df_corrected.reset_index()
service_df_corrected['YearMonth'] = service_df_corrected['YearMonth'].astype(str)

Plot du nopbre d'antennes en service par mois

In [23]:
fig_corrected = px.line(
    service_df_corrected, 
    x='YearMonth', 
    y='Antennas in Service', 
    title='Nombre d\'antennes en service par mois',
    markers=True,
    labels={'YearMonth': 'Mois', 'Antennas in Service': 'Nombre d\'antennes en service'}
)

fig_corrected.update_layout(
    xaxis_title='Mois',
    yaxis_title='Nombre d\'antennes en service',
    xaxis_tickangle=-45
)

output_file_corrected = 'antennas_in_service.html'
fig_corrected.write_html(output_file_corrected)